<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/fine-tuning-experiments/askpatient/askApatient_0_tuning_healthnews.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os

In [0]:
base_dir = 'medical-concept-normalization'

In [0]:
os.mkdir(base_dir)

In [0]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [0]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   287  100   287    0     0    986      0 --:--:-- --:--:-- --:--:--   982
bash: line 1: syntax error near unexpected token `newline'
bash: line 1: `<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">'


In [0]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
Unpacking objects: 100% (9/9), done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9


In [0]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [0]:
path = Path('medical-concept-normalization/data_collection')

In [0]:
# training data set

train_csv = path/'AskAPatient.fold-0.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [0]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-0.validation.csv",header=None)
valid.head()

,0,1
0,0,fear
1,2,heel pain
2,4,purple sploches around waist
3,7,goiter
4,8,ARTHRITIS


In [0]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-0.test.csv",header=None)
test.head()

,0,1
0,0,'scared' feeling
1,2,heel pain
2,3,cracking sensations in my joints
3,5,fast and slow heartbeat
4,5,increased heart rate


In [0]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'fine-tuning_language_model_healthnews_data.csv')

In [0]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [0]:
data_clas.add_test(test,label=0)


In [0]:
data_lm.save()
data_clas.save()

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:292: UserWarning: Couldn't retrieve source code for container of type CrossEntropyLoss. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [0]:
data_lm.train_ds.vocab.itos

In [0]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [0]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [0]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,6.933066,5.830900,0.144252,02:03


In [0]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,5.773152,5.286707,0.187165,02:03


In [0]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,5.401721,5.239043,0.192243,02:03


In [0]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,5.343619,5.190048,0.195982,02:03


In [0]:
learn.save_encoder('healthnews_fold0_first')

In [0]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('healthnews_fold0_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.758937,4.403602,0.310059,01:06


In [0]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.933096,4.061912,0.344379,01:21


In [0]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.590148,3.845391,0.369231,03:25


In [0]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.375410,3.658915,0.401183,03:33
1,3.187513,3.617022,0.408284,03:32


In [0]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.042000,3.459264,0.423669,03:23
1,2.995457,3.442854,0.418935,03:28


In [0]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.891451,3.406882,0.423669,03:25
1,2.753837,3.357276,0.437870,03:08


In [0]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.741826,3.317585,0.442604,03:26
1,2.678221,3.349233,0.447337,03:11


In [0]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.594691,3.291588,0.447337,03:29
1,2.474464,3.195058,0.446154,03:32


In [0]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.475619,3.216632,0.430769,03:27
1,2.480600,3.211527,0.442604,03:36
2,2.423870,3.181725,0.450888,03:05
3,2.308194,3.165788,0.453254,03:20


In [0]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.262378,3.177971,0.447337,03:34
1,2.317091,3.182335,0.453254,03:38
2,2.258620,3.186012,0.453254,03:27
3,2.098751,3.142022,0.457988,03:39


In [0]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.216715,3.204795,0.456805,03:25
1,2.360314,3.214164,0.468639,03:17
2,2.248865,3.225822,0.463905,03:10
3,2.172781,3.198708,0.457988,03:31


In [0]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.127084,3.229606,0.461538,03:21
1,2.287502,3.336414,0.463905,03:13
2,2.244742,3.311466,0.460355,03:17
3,2.071113,3.233939,0.468639,03:16


In [0]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.076073,3.232357,0.462722,03:28
1,2.145139,3.329790,0.460355,03:07
2,2.148004,3.330918,0.474556,03:34
3,2.057697,3.273148,0.471006,03:13


In [0]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.100601,3.366919,0.480473,03:29
1,2.129552,3.366944,0.459172,03:29
2,2.063259,3.326556,0.462722,03:24
3,1.917510,3.302889,0.472189,03:30


In [0]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.077312,3.408326,0.468639,03:31
1,2.128661,3.390277,0.468639,03:17
2,2.100440,3.433096,0.474556,03:15
3,2.110824,3.403623,0.469822,03:11


In [0]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.115690,3.474863,0.465089,03:33
1,2.190073,3.519124,0.465089,03:12
2,2.148184,3.384346,0.463905,03:22
3,1.895561,3.321220,0.465089,03:23


In [0]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.057747,3.442479,0.462722,03:17
1,2.090312,3.441307,0.467456,03:11
2,2.029938,3.389592,0.472189,03:13
3,2.058307,3.443530,0.472189,03:13


In [0]:
# save the best model

learn.save_encoder('ask-0-tuning-healthnews')

# Part three: Predict on the test dataset

In [0]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [0]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

867
867
[780, 2, 470, 334, 5, 8, 8, 8, 8, 8, 8, 8, 597, 360, 49, 49, 555, 27, 27, 27, 29, 289, 2, 30, 31, 584, 1028, 1035, 34, 289, 896, 289, 289, 289, 289, 896, 176, 44, 753, 49, 49, 49, 49, 49, 1035, 49, 1028, 1028, 1028, 1028, 1028, 1028, 1028, 1028, 753, 1028, 265, 61, 877, 877, 491, 228, 228, 182, 228, 228, 228, 1028, 1028, 1028, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 228, 713, 182, 864, 1028, 1028, 80, 80, 80, 80, 80, 80, 80, 80, 80, 108, 89, 427, 137, 49, 265, 92, 753, 94, 1028, 1028, 1028, 1028, 1028, 1028, 1028, 318, 408, 1035, 737, 100, 780, 644, 100, 100, 214, 214, 214, 100, 805, 291, 265, 108, 108, 108, 209, 363, 111, 111, 111, 265, 265, 579, 117, 446, 1028, 1028, 1028, 125, 1028, 1028, 125, 125, 125, 125, 125, 125, 125, 125, 125, 343, 289, 131, 1035, 447, 447, 344, 567, 37, 151, 151, 182, 284, 305, 158, 165, 165, 753, 359, 1028, 291, 289, 289, 289, 289, 289, 289, 289, 524, 111, 289, 182, 182, 

In [0]:
correct_num = 0

for i in range(0, 866):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

240
0.2768166089965398
